## Problem 1: Basic concentric frame transforms
Most of the frames have the same origin. The code in chap2/transforms.py calculates the rotation matrices used to transform between these frames. Correctly implement the following functions:
* rot_x: calculate elementary rotation matrix about x-axis
* rot_y: calculate elementary rotation matrix about y-axis
* rot_z: calculate elementary rotation matrix about z-axis
* rot_v_to_v1: calculates the rotation from frame v to v1
* rot_v1_to_v2: calculates the rotation from frame v1 to v2
* rot_v2_to_b: calculates the rotation from v2 to body frame
* rot_b_to_s: calculates the rotation from body to stability frame
* rot_s_to_w: calculates the rotation from stability to wind frame

*Hint:* You should only compute the cosine and sine of the angle in *rot_x*, *rot_y*, and *rot_z*. All the remaining functions should call those functions (i.e., one line change from what they currently are)

Use these function to compute the following. Assume that $\psi = \frac{\pi}{4}$, $\theta = 0.3$, $\phi = 0.25$, $\alpha = 0.1$, and $\beta = 0.15$. Display the results in the exported pdf.
* Compute $p_1^{v1}$ given $p_1^v = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$
* Compute $p_1^{w}$
* Compute $p_2^{s}$ given $p_2^{v2} = \begin{bmatrix} 4 \\ 5 \\ 6 \end{bmatrix}$
* Compute $p_2^v$



In [5]:
%autoreload 1
import numpy as np # Imports the numpy library and creates the alias np
from IPython.display import display # Used to display variables nicely in Jupyter
from mav_sim.chap2.transforms import rot_v_to_v1, rot_v1_to_v2, rot_v2_to_b, rot_b_to_s, rot_s_to_w

# Calculate the required rotation matrices
psi = np.pi/4
theta = 0.3
phi = 0.25
alpha = 0.1
beta = 0.15
R_v_to_v1 = rot_v_to_v1(psi)
R_v1_to_v2 = rot_v1_to_v2(theta)
R_v2_to_b = rot_v2_to_b(phi)
R_b_to_s = rot_b_to_s(alpha)
R_s_to_w = rot_s_to_w(beta)

# Initialize p1 and p2
p_1_v = np.array([[1],[2],[3]])
p_2_v2 = np.array([[4],[5],[6]])

# Calculate p_1^v1
p_1_v1 = R_v_to_v1 @ p_1_v
print('p_1^v1 = ')
display(p_1_v1)

# Calculate p_1^w
p_1_w = R_s_to_w @ R_b_to_s @ R_v2_to_b @ R_v1_to_v2 @ R_v_to_v1 @ p_1_v
print('p_1^w = ')
display(p_1_w)

# Calculate p_2^s
p_2_s = R_s_to_w @ R_b_to_s @ R_v2_to_b @ p_2_v2
print('p_2^s = ')
display(p_2_s)

# Calculate p_2^v
R_v2_to_v1 = R_v1_to_v2.T
R_v1_to_v = R_v_to_v1.T
p_2_v = R_v1_to_v @ R_v2_to_v1 @ p_2_v2
print('p_2^v = ')
display(p_2_v)


Gets here
p_1^v1 = 


array([[2.12132034],
       [0.70710678],
       [3.        ]])

p_1^w = 


array([[1.66990864],
       [1.31449478],
       [3.07953058]])

p_2^s = 


array([[5.33287   ],
       [5.59487616],
       [4.15425786]])

p_2^v = 


array([[0.42035179],
       [7.4914196 ],
       [4.54993811]])




## Problem 2: Compound rotation matrices
The transform from the vehicle frame to the body frame can be written as a compound of three rotation matrices (and so can the inverse transform). However, these matrices are used so often that it is nice to avoid multiplying these three matrices each time the transform is needed. 

Implement the following functions:

* rot_v_to_b: calculates the rotation from vehicle to body frame
* rot_b_to_v: calculates the rotation from body frame to vehicle frame

*Hint:* You really only need to implement one of them and then use a transpose for the other

Using the same values as above, show that your implementation produces the same rotation matrices as three elementary matrices multiplied together. Display the difference in the exported pdf.


In [2]:
from mav_sim.chap2.transforms import rot_v_to_b, rot_b_to_v

# Calculate the rotation matrices as compound rotation matrices (i.e., matrix multiplication)
R_v_to_b_comp = R_v2_to_b @ R_v1_to_v2 @ R_v_to_v1
R_b_to_v_comp = R_v_to_b_comp.T

# Calculate the rotation matrices using the functions
R_v_to_b_func = rot_v_to_b(psi, theta, phi)
R_b_to_v_func = rot_b_to_v(psi, theta, phi)

# Calculate and display the difference
R_v_to_b_diff = R_v_to_b_comp - R_v_to_b_func
print("R_v_to_b_diff")
display(R_v_to_b_diff)

R_b_to_v_diff = R_b_to_v_comp - R_b_to_v_func
print("R_b_to_v_diff")
display(R_b_to_v_diff)


R_v_to_b_diff


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.55111512e-17,  0.00000000e+00,  0.00000000e+00]])

R_b_to_v_diff


array([[ 0.00000000e+00,  0.00000000e+00, -5.55111512e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

## Problem 3: Tranform to vehicle frame
Converting to and from the inertial frame requires translation. Implement the following functions:
* trans_i_to_v: transforms a point from inertial frame to the vehicle frame
* trans_v_to_i: transforms a point from vehicle frame to the inertial frame
* trans_i_to_b: transforms a point from inertial frame to the body frame
* trans_b_to_i: transforms a point from the body frame to the inertial frame

Note that the transform between inertial and the vehicle frame is purely translational. The transform between the vehicle and body frame is purely rotational. Thus, you can use the functions already implemented to make the *trans_i_to_b* and *trans_b_to_i* functions quite simple.

Given that the UAV is in the position $p_n = 1$, $p_e = 2$, and $p_d = 3$ with the angles defined as before, transform the following points to the body frame using the implemented functions:

$p_3^i = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

$p_4^i = \begin{bmatrix} 3 \\ 2 \\ 1\end{bmatrix}$

Transform the following point in the body frame to the inertial frame

$p_5^b = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

Transform the following point in the wind frame to the inertial frame

$p_6^w = \begin{bmatrix} 1 \\ 2 \\ 3\end{bmatrix}$

Display the results in the exported pdf.

In [3]:
from mav_sim.chap2.transforms import trans_i_to_b, trans_b_to_i

# Create the pose of the aircraft
class Pose:
    def __init__(self) -> None:
        self.north: float     = 1.      # north position
        self.east: float      = 2.      # east position
        self.altitude: float  = 3.      # altitude
        self.phi: float       = phi     # roll angle
        self.theta: float     = theta   # pitch angle
        self.psi: float       = psi     # yaw angle
pose = Pose()

# Initialize the points
p_3_i = np.array([[1],[2],[3]])
p_4_i = np.array([[3],[2],[1]])
p_5_b = np.array([[1],[2],[3]])
p_6_w = np.array([[1],[2],[3]])

# Calculate p_3^b
p_3_b = trans_i_to_b(pose, p_3_i)
print("p_3^b = ")
display(p_3_b)

# Calculate p_4_b
p_4_b = trans_i_to_b(pose, p_4_i)
print("p_4^b = ")
display(p_4_b)

# Calcualte p_5^i
p_5_i = trans_b_to_i(pose, p_5_b)
print("p_5^i")
display(p_5_i)

# Calculate p_6^b
R_s_to_b = R_b_to_s.T
R_w_to_s = R_s_to_w.T
p_6_b = R_s_to_b @ R_w_to_s @ p_6_w

# Calculate p_6^i
p_6_i = trans_b_to_i(pose, p_6_b)
print("p_6^i")
display(p_6_i)


p_3^b = 


array([[-1.77312124],
       [ 1.41812418],
       [ 5.55382435]])

p_4^b = 


array([[ 0.16896899],
       [-0.32143576],
       [ 4.45736789]])

p_5^i


array([[ 1.54090053],
       [ 4.2317526 ],
       [-0.04589997]])

p_6^i


array([[1.06667346],
       [3.91266605],
       [0.21516146]])

## Simple code checking
The following code does not need to change. It should just be used as a sanity check so that you know the code is implemented properly. The output should not have any lines reading `Failed test!`

In [4]:
from mav_sim.unit_tests.ch2_transforms_tests import run_all_tests
run_all_tests()

Starting rot_x test

Calculated output:
[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00  1.2246468e-16]
 [ 0.0000000e+00 -1.2246468e-16 -1.0000000e+00]]
Expected output:
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
Passed test
Calculated output:
[[ 1.          0.          0.        ]
 [ 0.          0.0707372   0.99749499]
 [ 0.         -0.99749499  0.0707372 ]]
Expected output:
[[ 1.          0.          0.        ]
 [ 0.          0.0707372   0.99749499]
 [ 0.         -0.99749499  0.0707372 ]]
Passed test
End of test

Starting rot_y test

Calculated output:
[[-1.0000000e+00  0.0000000e+00 -1.2246468e-16]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [ 1.2246468e-16  0.0000000e+00 -1.0000000e+00]]
Expected output:
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
Passed test
Calculated output:
[[ 0.0707372   0.         -0.99749499]
 [ 0.          1.          0.        ]
 [ 0.99749499  0.          0.0707372 ]]
Expected output:
[[ 0.0707372   0.         -0.99749499]
 [ 0.   